# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [1]:
# Check which Python environment Jupyter is using
import sys
print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"Python path: {sys.path}")

Python version: 3.11.10 (main, Sep  7 2024, 01:03:31) [Clang 16.0.0 (clang-1600.0.26.3)]
Python executable: /opt/homebrew/opt/python@3.11/bin/python3.11
Python path: ['/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.10/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/home/Library/Python/3.11/lib/python/site-packages', '/opt/homebrew/lib/python3.11/site-packages']


In [2]:
# Add the site-packages directory to the Python path
import sys
sys.path.append('/opt/homebrew/lib/python3.11/site-packages')

# Now try importing the packages
import requests
from bs4 import BeautifulSoup
print("Successfully imported packages!")

Successfully imported packages!


In [3]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [4]:
# Add these artists to your dictionary
artists = {
    'robyn': "https://www.azlyrics.com/r/robyn.html",
    'cher': "https://www.azlyrics.com/c/cher.html",
    'cory_asbury': "https://www.azlyrics.com/c/coryasbury.html",
    'hillsong_united': "https://www.azlyrics.com/h/hillsongunited.html"
} 
# You can replace these with your chosen artists

# Set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())  # Be nice to the server

    # extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, 'html.parser')
    # Find the div containing song links
    album_list = soup.find('div', {'class': 'album-list'})
    if album_list:
        # Get all links to song lyrics
        links = album_list.find_all('a', href=True)
        for link in links:
            if '/lyrics/' in link['href']:
                # Add the full URL to our dictionary
                lyrics_pages[artist].append('https://www.azlyrics.com' + link['href'])

# Verify we have enough songs for each artist
for artist, lp in lyrics_pages.items():
    assert(len(set(lp)) > 20)
    print(f"For {artist} we have {len(lp)} songs.")
    print(f"The full pull will take for this artist will take {round(len(lp)*10/3600,2)} hours.")

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: <!-- Delete this comment and put your answer here. --> 


In [6]:
# Set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())  # Be nice to the server

    # extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, 'html.parser')
    # Find the div containing song links
    album_list = soup.find('div', {'class': 'album-list'})
    if album_list:
        # Get all links to song lyrics
        links = album_list.find_all('a', href=True)
        for link in links:
            if '/lyrics/' in link['href']:
                # Add the full URL to our dictionary
                lyrics_pages[artist].append('https://www.azlyrics.com' + link['href'])

# Verify we have enough songs for each artist
for artist, lp in lyrics_pages.items():
    assert(len(set(lp)) > 20)
    print(f"For {artist} we have {len(lp)} songs.")
    print(f"The full pull will take for this artist will take {round(len(lp)*10/3600,2)} hours.")

## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [9]:
def generate_filename_from_link(link):
    if not link:
        return None
    
    # drop the http or https and the html
    name = link.replace("https://", "").replace("http://", "")
    name = name.replace(".html", "")
    name = name.replace("www.azlyrics.com/lyrics/", "")
    
    # Replace useless characters with UNDERSCORE
    name = name.replace(".", "_").replace("/", "_")
    
    # tack on .txt
    name = name + ".txt"
    
    return name

In [10]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [12]:
# Make the lyrics folder
import shutil
if os.path.isdir("lyrics"):
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

start = time.time()
total_pages = 0

for artist in lyrics_pages:
    # 1. Build a subfolder for the artist
    artist_dir = os.path.join("lyrics", artist)
    os.mkdir(artist_dir)
    
    print(f"Starting to scrape lyrics for {artist}...")
    
    # 2. Iterate over the lyrics pages (limit to 25 songs per artist for time)
    for page_url in lyrics_pages[artist][:25]:
        try:
            # 3. Request the lyrics page
            response = requests.get(page_url)
            time.sleep(5 + 10*random.random())  # Be nice to the server
            
            # 4. Extract the title and lyrics from the page
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find the title
            title_div = soup.find('div', {'class': 'ringtone'})
            title = "Unknown Title"
            if title_div and title_div.find_previous('b'):
                title = title_div.find_previous('b').text.strip()
            
            # Find the lyrics
            lyrics_div = soup.find('div', {'class': None, 'id': None}, 
                                  attrs={'style': None})
            if lyrics_div:
                lyrics = lyrics_div.text.strip()
                
                # 5. Write out the title, two returns, and the lyrics
                filename = generate_filename_from_link(page_url)
                with open(os.path.join(artist_dir, filename), 'w', encoding='utf-8') as f:
                    f.write(title + '\n\n' + lyrics)
                
                total_pages += 1
                print(f"Saved lyrics for {artist} - {title}")
        except Exception as e:
            print(f"Error processing {page_url}: {e}")

print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")
print(f"Total pages scraped: {total_pages}")

Total run time was 0.0 hours.
Total pages scraped: 0


In [13]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders:
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files:
        with open("lyrics/" + artist + "/" + f_name, encoding='utf-8') as infile:
            artist_words.extend(words(infile.read()))
            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")
    
    # Print the most common words
    word_counts = Counter(artist_words)
    print(f"Most common words for {artist}:")
    for word, count in word_counts.most_common(20):
        if len(word) > 2:  # Filter out very short words
            print(f"  {word}: {count}")
    print("\n")

In [14]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.0 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [ ]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [ ]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


In [16]:
! install matplotlib

usage: install [-bCcpSsUv] [-f flags] [-g group] [-m mode] [-o owner]
               [-M log] [-D dest] [-h hash] [-T tags]
               [-B suffix] [-l linkflags] [-N dbdir]
               file1 file2
       install [-bCcpSsUv] [-f flags] [-g group] [-m mode] [-o owner]
               [-M log] [-D dest] [-h hash] [-T tags]
               [-B suffix] [-l linkflags] [-N dbdir]
               file1 ... fileN directory
       install -dU [-vU] [-g group] [-m mode] [-N dbdir] [-o owner]
               [-M log] [-D dest] [-h hash] [-T tags]
               directory ...


In [17]:
# Optional: Add visualization
import matplotlib.pyplot as plt
import numpy as np

# Create a dictionary to store word counts for each artist
artist_word_counts = {}

for artist in artist_folders:
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    artist_words = []
    for f_name in artist_files:
        with open("lyrics/" + artist + "/" + f_name, encoding='utf-8') as infile:
            artist_words.extend(words(infile.read()))
    
    # Store word counts for this artist
    artist_word_counts[artist] = Counter(artist_words)

# Plot the most common words for each artist
plt.figure(figsize=(15, 10))

for i, (artist, word_counts) in enumerate(artist_word_counts.items()):
    # Get the 10 most common words with length > 2
    common_words = [(word, count) for word, count in word_counts.most_common(50) if len(word) > 2][:10]
    words_list, counts = zip(*common_words)
    
    plt.subplot(2, 2, i+1)
    plt.barh(words_list, counts)
    plt.title(f"Most Common Words for {artist}")
    plt.xlabel("Count")
    plt.tight_layout()

plt.savefig("artist_word_comparison.png")
plt.show()

ModuleNotFoundError: No module named 'matplotlib'